In [1]:
# Dependencies as needed
import gmaps
import matplotlib.pyplot as plt
from citipy import citipy
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import requests
from census import Census
import gmaps 
import time
from us import states
import json


In [2]:
combined_data = pd.read_csv("Census_Zillow_Combined_CA.csv")
combined_data.head()

,Zipcode,City,State,CountyName,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,average rent,Median Home Value,Number of House Build in year,Avg_HomeValue
0,90001,Florence-Graham,CA,Los Angeles,57942.0,28.0,34323.0,11294.0,18902.0,32.622277,915.0,272200.0,1950.0,2.999500e+05
1,90002,Los Angeles,CA,Los Angeles,51826.0,27.5,32520.0,11212.0,18302.0,35.314321,902.0,248200.0,1951.0,2.946667e+05
2,90003,Los Angeles,CA,Los Angeles,70208.0,28.0,31878.0,10611.0,24556.0,34.976071,958.0,268500.0,1951.0,3.002417e+05
3,90004,Los Angeles,CA,Los Angeles,63095.0,35.5,43180.0,29194.0,12919.0,20.475473,1044.0,838000.0,1946.0,1.168492e+06
4,90005,Los Angeles,CA,Los Angeles,39338.0,33.9,31485.0,20265.0,11520.0,29.284661,943.0,672600.0,1947.0,7.051917e+05


In [3]:
api_key = "288538c4c4aa958caf000f4ac70ef2f0"
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key + "&q="

combined_data["Latitude"] = ""
combined_data["Longitude"] = ""

for index, row in combined_data.iterrows():
    city = row["City"]+",us"
    response = requests.get(url + city).json()
    
    try:
        combined_data.loc[index, "Latitude"] = response["coord"]["lat"]
        combined_data.loc[index, "Longitude"] = response["coord"]["lon"]     
    except:
        pass
    
combined_data.head()

KeyboardInterrupt: 

In [11]:
# limited the lat and lng within California  "min_lat": 32.5121,"max_lat": 42.0126,"min_lng": -124.6509,"max_lng": -114.1315
combined_data = combined_data[combined_data["Latitude"] != ""]
combined_data = combined_data[combined_data["Latitude"] >= 32.5121]
combined_data = combined_data[combined_data["Longitude"] <= -114.1315]
combined_data = combined_data[combined_data["Longitude"] >= -124.6509]
combined_data.count()

Zipcode                          1072
City                             1072
State                            1072
CountyName                       1072
Population                       1072
Median Age                       1072
Household Income                 1072
Per Capita Income                1072
Poverty Count                    1072
Poverty Rate                     1072
average rent                     1072
Median Home Value                1072
Number of House Build in year    1072
Avg_HomeValue                    1072
Latitude                         1072
Longitude                        1072
dtype: int64

In [12]:
combined_data_df = combined_data.groupby("City")
#560 unique cities in CA
len(combined_data_df)


559

In [13]:
nom=Nominatim(user_agent = "my-application")

In [14]:
# Example format
nom.geocode("San Francisco,CA 94114")
# Outputs city abbreviation, state, zipcode, country, (lat,long,geocode response)

Location(SF, California, 94114, United States of America, (37.7614032, -122.435242, 0.0))

In [15]:
df = combined_data[["Zipcode","City","Latitude","Longitude"]]
unique_cities = df["City"].unique()
temp_lat = []
temp_long = []

for city in unique_cities:
    temp_lat.append(df[df["City"] == city].Latitude)
    temp_long.append(df[df["City"] == city].Longitude)


# Check    
#print(len(cities_only))
#print(len(temp_lat))
#print(len(temp_long))


# Work around for groupby not working?
condensed_dict = {"City" : unique_cities,
                  "Latitude" : temp_lat,
                  "Longitude" : temp_long
                 }
condensed_df = pd.DataFrame(condensed_dict)
condensed_df.head()

,City,Latitude,Longitude
0,Florence-Graham,"0 33.97 Name: Latitude, dtype: object","0 -118.25 Name: Longitude, dtype: object"
1,Los Angeles,1 34.05 2 34.05 3 34.05 4 ...,1 -118.24 2 -118.24 3 -118.24 4 ...
2,East Los Angeles,"18 34.02 50 34.02 Name: Latitude, dtype:...","18 -118.17 50 -118.17 Name: Longitude, dty..."
3,Ladera Heights,"45 33.99 Name: Latitude, dtype: object","45 -118.38 Name: Longitude, dtype: object"
4,West Hollywood,"56 34.09 Name: Latitude, dtype: object","56 -118.36 Name: Longitude, dtype: object"


In [10]:
# Get zip code for the states to setup geopy address
zipcodes = []
running = True

for city in combined_data["City"]:   
    if running:
        zipcodes.append(combined_data[combined_data["City"] == city].Zipcode)
        running = False
    else:
        running = True
len(zipcodes)
        
    

536

In [14]:
condensed_df["Address"] = condensed_df["City"] + ", CA "

In [17]:
#????????????
condensed_df

,City,Latitude,Longitude
0,Florence-Graham,"0 33.97 Name: Latitude, dtype: object","0 -118.25 Name: Longitude, dtype: object"
1,Los Angeles,1 34.05 2 34.05 3 34.05 4 ...,1 -118.24 2 -118.24 3 -118.24 4 ...
2,East Los Angeles,"18 34.02 50 34.02 Name: Latitude, dtype:...","18 -118.17 50 -118.17 Name: Longitude, dty..."
3,Ladera Heights,"45 33.99 Name: Latitude, dtype: object","45 -118.38 Name: Longitude, dtype: object"
4,West Hollywood,"56 34.09 Name: Latitude, dtype: object","56 -118.36 Name: Longitude, dtype: object"
5,Bell,"59 33.98 Name: Latitude, dtype: object","59 -118.19 Name: Longitude, dtype: object"
6,Beverly Hills,60 34.07 61 34.07 62 34.07 Name: Lati...,60 -118.4 61 -118.4 62 -118.4 Name: Long...
7,Compton,63 33.89 64 33.89 65 33.89 Name: Lati...,63 -118.23 64 -118.23 65 -118.23 Name: L...
8,Culver City,"66 34.02 67 34.02 Name: Latitude, dtype:...","66 -118.4 67 -118.4 Name: Longitude, dtype..."
9,Downey,68 33.94 69 33.94 70 33.94 Name: Lati...,68 -118.13 69 -118.13 70 -118.13 Name: L...


In [33]:
# unique combinations of latitudes and longitudes

combined_data_lat = combined_data.Latitude
combined_data_long = combined_data.Longitude

coords = [(x,y) for (x,y) in zip(combined_data_lat, combined_data_long)]
unique_coords = []
for coord in coords:
    if coord not in unique_coords:
        unique_coords.append(coord)
    else:
        pass
unique_coords
# len(unique_coords) outputs 559

[(33.97, -118.25),
 (34.05, -118.24),
 (34.02, -118.17),
 (33.99, -118.38),
 (34.09, -118.36),
 (33.98, -118.19),
 (34.07, -118.4),
 (33.89, -118.23),
 (34.02, -118.4),
 (33.94, -118.13),
 (33.91, -118.38),
 (33.89, -118.31),
 (33.91, -118.35),
 (33.86, -118.4),
 (33.98, -118.23),
 (33.89, -118.35),
 (33.93, -118.21),
 (33.9, -118.4),
 (33.79, -118.4),
 (33.75, -118.37),
 (33.85, -118.39),
 (33.95, -118.2),
 (34.09, -118.6),
 (33.96, -118.36),
 (34.02, -118.49),
 (33.84, -118.34),
 (33.82, -118.29),
 (33.97, -118.03),
 (33.94, -118.03),
 (33.98, -118.07),
 (33.87, -118),
 (33.85, -118.05),
 (33.93, -117.94),
 (33.91, -118.01),
 (34.02, -118.11),
 (33.91, -118.08),
 (33.98, -118.1),
 (33.95, -118.07),
 (33.86, -118.05),
 (33.88, -118.12),
 (33.85, -118.12),
 (33.83, -118.07),
 (33.8, -118.32),
 (33.8, -118.08),
 (33.9, -118.17),
 (33.74, -118.11),
 (33.83, -118.25),
 (33.8, -118.17),
 (34.19, -118.14),
 (34.14, -118.04),
 (34.15, -117.97),
 (34.14, -117.98),
 (34.2, -118.19),
 (34.15, -

In [4]:
from pyzipcode import ZipCodeDatabase
zcdb = ZipCodeDatabase()
#Example zipcode 91776
zips = [z.zip for z in zcdb.get_zipcodes_around_radius('91776', 5)]

#zip to lat long with pyzipcode
zipcode = zcdb[zips[0]]
print(zipcode.latitude)
print(zipcode.longitude)

#Geopy reverse did not work? Use citipy instead
#lat long to city with citipy
city = citipy.nearest_city(zipcode.latitude,zipcode.longitude)
city.city_name

34.0245
-118.1561


'east los angeles'

In [5]:
cities_nearby = []
for i in zips:
    zipcode = zcdb[i]
    city = citipy.nearest_city(zipcode.latitude,zipcode.longitude)
    if city.city_name not in cities_nearby:
        cities_nearby.append(city.city_name)
    else:
        pass
cities_nearby

['east los angeles',
 'commerce',
 'whittier',
 'montebello',
 'pico rivera',
 'altadena',
 'arcadia',
 'temple city',
 'sierra madre',
 'south pasadena',
 'pasadena',
 'san marino',
 'el monte',
 'south el monte',
 'monterey park',
 'rosemead',
 'san gabriel',
 'alhambra']

In [22]:
#Geopy distances
from geopy import Nominatim
from geopy import distance
d = distance.distance
g = Nominatim(user_agent="my_application")

for city in cities_nearby:
    zipcode1 = zcdb['91776']
    city1 = (zipcode1.latitude, zipcode1.longitude)
    city2 = (combined_data[combined_data["City" == city].Latitude,], combined_data[combined_data["City" == city].Longitude,])
    
    print(distance.distance(city1, city2).miles)

KeyError: False

In [ ]:
#find nearest 5 cities, chart housing sales, housing prices for the past years using zillow housing pricesfor the past years and compare
